In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern,DotProduct
#import grid search
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#import make_scorer
from sklearn.metrics import make_scorer
import torch
import esm
#import pearsonr
from scipy.stats import pearsonr
import tqdm
import random
#import spearmanr
from scipy.stats import spearmanr
#import r2_score
from sklearn.metrics import r2_score
from copy import deepcopy
#close warnings
import warnings
warnings.filterwarnings("ignore")
##############
sns.set_context('paper', font_scale=1.2)
sns.set_style('white')
#sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_style({'font.family':'serif','font.serif':'Times New Roman'})
sns.despine()
plt.tight_layout()

In [ ]:
wt = "MTYKLIINGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"
sites = ["V39","D40","G41","V54"]

In [ ]:
RG = np.random.default_rng(42)
def pi(y_pre,y_std):
    from scipy.stats import norm
    y_max = y_pre.max()
    probs = norm.cdf((y_pre - y_max) / (y_std+1E-9))
    return probs
def ucb(y,y_std,beta=2):
    # _y = y/y.max()
    # _std = y_std/y_std.max()
    return y + beta * y_std

def random(Y_mean: np.ndarray) -> np.ndarray:
    """Random acquistion score"""
    return RG.random(len(Y_mean))


class Acquisiton:
    def __init__(self, pool):
        self.pool = pool

    def random(self, sample_size=1000):

        return self.pool.sample(n=sample_size)# random_state=42)

    def entropy_sample(self, props, sample_size=500):
        from scipy.stats import entropy

        # props = props.reshape((-1, 1))
        # props = np.hstack((1 - props, props))
        entropies = entropy(props, axis=1, base=2)
        _pool = self.pool
        _pool["entropy"] = entropies
        _pool.sort_values("entropy", inplace=True)
        return _pool.iloc[-sample_size:]
    
    def ucb_sample(self,y_pre,y_std,sample_size=10,beta=2):
        u = ucb(y=y_pre,y_std=y_std,beta=beta)
        _pool = self.pool
        _pool["u"] = u
        _pool.sort_values("u", inplace=True)
        return _pool.iloc[-sample_size:]
    def pi_sample(self,y_pre,y_std,sample_size=10):
        u = pi(y_pre=y_pre,y_std=y_std)
        _pool = self.pool
        _pool["u"] = u
        _pool.sort_values("u", inplace=True)
        return _pool.iloc[-sample_size:]


In [ ]:
amino_acids = [
    'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R',
    'S', 'T', 'V', 'W', 'Y'
]

def get_esm(seq):
    data = [('0', seq.strip())]

    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    if torch.cuda.is_available():
        batch_tokens = batch_tokens.to(device="cuda", non_blocking=True)
    batch_tokens = batch_tokens.reshape((1, -1))
    # Extract per-residue representations
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=False)
    token_representations = results["representations"][33]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1:tokens_len -
                                                              1].mean(0))
    return sequence_representations[0].cpu()

#ref: Biswas, Surojit, et al. "Low-N protein engineering with data-efficient deep learning." Nature methods 18.4 (2021): 389-396.

def run_DE_trajectories(s_wt,
                        Model,
                        num_iterations,
                        num_trajectories,
                        T=0.01,
                        trust_radius=12,
                        fixed_sites=None):

    s_records = []  # initialize list of sequence records
    y_records = []  # initialize list of fitness score records

    for i in tqdm.tqdm(range(num_trajectories)): #iterate through however many mutation trajectories we want to sample
        s_traj, y_traj = directed_evolution(
            s_wt=s_wt,
            num_iterations=num_iterations,
            Model=Model,
            T=T,
            trust_radius=trust_radius,
            fixed_sites=fixed_sites
        )  # call the directed evolution function, outputting the trajectory sequence and fitness score records

        s_records.append(
            s_traj
        )  # update the sequence trajectory records for this full mutagenesis trajectory
        y_records.append(
            y_traj
        )  # update the fitness trajectory records for this full mutagenesis trajectory
    s_records = np.array(s_records)
    y_records = np.array(y_records)

    # plt.clf()
    # fig = plt.figure(figsize=(10,6))
    # plt.plot(np.transpose(y_records[:,:,0])) # plot the changes in fitness for all sampled trajectories
    # plt.ylabel('Predicted Fitness')
    # plt.xlabel('Mutation Trial Steps')
    # plt.show() # show the plot :)

    return s_records, y_records


def directed_evolution(
    s_wt,
    num_iterations,
    Model,
    T=0.01,
    trust_radius=12,
    fixed_sites=None
):  # input = (wild-type sequence, number of mutation iterations, "temperature")

    s_traj = [
    ]  # initialize an array to keep records of the protein sequences for this trajectory
    y_traj = [
    ]  # initialize an array to keep records of the fitness scores for this trajectory

    mut_loc_seed = random.randint(
        0, len(s_wt)
    )  # randomely choose the location of the first mutation in the trajectory
    s, new_mut_loc = mutate_sequence(
        s_wt, (np.random.poisson(2) + 1),
        mut_loc_seed,
        fixed_sites=fixed_sites
    )  # initial mutant sequence for this trajectory, with m = Poisson(2)+1 mutations

    x = get_esm(s).reshape((1, -1))

    y = Model.predict(
        x
    )  # predicted fitness score for the initial mutant sequence for this trajectory

    # iterate through the trial mutation steps for the directed evolution trajectory
    for i in range(num_iterations):
        mu = np.random.uniform(
            1, 2.5
        )  # "mu" parameter for poisson function: used to control how many mutations to introduce
        m = np.random.poisson(
            mu -
            1) + 1  # how many random mutations to apply to current sequence
        s_new, new_mut_loc = mutate_sequence(
            s, m, new_mut_loc, fixed_sites=fixed_sites
        )  # new trial sequence, produced from "m" random mutations
        if trust_radius:
            truths = (np.array(list(s_new)) == np.array(list(s_wt)))
            num_mut = np.count_nonzero(~truths)
            if num_mut <= trust_radius:
                x_new = get_esm(s_new).reshape((1, -1))
                y_new = Model.predict(
                    x_new)  # new fitness value for trial sequence
            else:
                y_new = -99999999
        else:
            x_new = get_esm(s_new).reshape((1, -1))
            y_new = Model.predict(
                x_new)  # new fitness value for trial sequence
        with warnings.catch_warnings():  # catching Overflow warning
            warnings.simplefilter("ignore")
            try:
                p = min(1,
                        np.exp((y_new - y) /
                               T))  # probability function for trial sequence
            except OverflowError:
                p = 1
        rand_var = random.random()

        if rand_var < p:  # metropolis-Hastings update selection criterion
            #print(str(new_mut_loc+1)+" "+s[new_mut_loc]+"->"+s_new[new_mut_loc])
            s, y = s_new, y_new  # if criteria is met, update sequence and corresponding fitness

        s_traj.append(
            s
        )  # update the sequence trajectory records for this iteration of mutagenesis
        y_traj.append(
            y
        )  # update the fitness trajectory records for this iteration of mutagenesis

    return s_traj, y_traj  # output = (sequence record for trajectory, fitness score recorf for trajectory)


def mutate_sequence(
    seq,
    m,
    prev_mut_loc,
    fixed_sites=None
):  # produce a mutant sequence (integer representation), given an initial sequence and the number of mutations to introduce ("m")

    for i in range(m):  #iterate through number of mutations to add
        if not fixed_sites:
            rand_loc = random.randint(prev_mut_loc - 8, prev_mut_loc +
                                      8)  # find random position to mutate
            while (rand_loc <= 0) or (rand_loc >= len(seq)):
                rand_loc = random.randint(prev_mut_loc - 8, prev_mut_loc + 8)

            rand_aa = random.randint(0,
                                     19)  # find random amino acid to mutate to
            seq = list(seq)
            seq[rand_loc] = amino_acids[
                rand_aa]  # update sequence to have new amino acid at randomely chosen position
            seq = ''.join(seq)
        else:
            positions = []
            for mu_site in fixed_sites:
                pos = int(mu_site[1:]) - 1
                if pos not in positions:
                    positions.append(pos)
            rand_loc = random.choice(positions)
            rand_aa = random.randint(0, 19)
            seq = list(seq)
            seq[rand_loc] = amino_acids[rand_aa]
            seq = ''.join(seq)
    return seq, rand_loc  # output the randomely mutated sequence

In [ ]:
#function report pearsonr,spearmanr,r2 print out
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import r2_score
def report(model,X_test,y_true):
    y_pred = model.predict(X_test)
    pearsonr_ = pearsonr(y_true, y_pred)
    spearmanr_ = spearmanr(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print('Pearsonr: %.3f, p-value: %.3f' % (pearsonr_[0], pearsonr_[1]))
    print('Spearmanr: %.3f, p-value: %.3f' % (spearmanr_[0], spearmanr_[1]))
    print('R2: %.3f' % (r2))
    plt.plot(y_true, y_pred, 'o')
    plt.plot(np.linspace(y_true.min(), y_true.max(), 100), np.linspace(y_true.min(), y_true.max(), 100), 'r--')


In [ ]:
def generate_focused_lib(wt,sites,model,num_iterations=30,num_trajectories=300,T=0.01,trust_radius=None):
    num_iterations = num_iterations
    num_trajectories = num_trajectories
    seqs, ys = run_DE_trajectories(s_wt=wt,
                                    Model=model,
                                    num_iterations=num_iterations,
                                    num_trajectories=num_trajectories,
                                    T=0.01,
                                    trust_radius=None,
                                    fixed_sites=sites)
    df_res = pd.DataFrame({
        'seq':
        seqs.ravel(),
        'y':
        ys.reshape((num_trajectories, num_iterations)).ravel()
    })
    return df_res

In [ ]:
df = pd.read_csv("gb1-paper/gb1_2016_149361_seq.csv")
df

In [ ]:
df1 = df[df.AACombo.str.fullmatch('.?DGV')]
df2 = df[df.AACombo.str.fullmatch('V.?GV')]
df3 = df[df.AACombo.str.fullmatch('VD.?V')]
df4 = df[df.AACombo.str.fullmatch('VDG.?')]
dftest = pd.concat([df1,df2,df3,df4])
dftest

In [ ]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
#model.eval()
if torch.cuda.is_available():
  model = model.cuda()
model.eval()  # disables dropout for deterministic results

In [ ]:
def get_esm(seq):
    data = [('0',seq.strip())]

    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    if torch.cuda.is_available():
      batch_tokens = batch_tokens.to(device="cuda", non_blocking=True)
    batch_tokens = batch_tokens.reshape((1,-1))
    # Extract per-residue representations
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=False)
    token_representations = results["representations"][33]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
    return sequence_representations[0].cpu()

In [ ]:
def fit_model(model,param_grid,X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    grid = GridSearchCV(model,param_grid,refit=True,cv=3,n_jobs=-1)
    grid.fit(X_train,y_train)
    print("score:",grid.best_score_)
    print("best_params:",grid.best_params_)
    print("#####train set#####")
    report(grid,X_train,y_train)
    print("#####test set#####")
    report(grid,X_test,y_test)
    grid.best_estimator_.fit(X,y)
    return grid.best_estimator_

In [ ]:
#svr hyperparameter tuning
svr_param_grid = {'C': [.01,0.1, 1, 10, 100,500, 1000, 5000,10000],
                'gamma': [10,100,1000,500,1, 0.1, 0.01, 0.001, 0.0001]}
#gpr hyperparameter tuning
gpr_param_grid = {'alpha': [1e-10,.01,0.1, 1, 10, 1e-8,1e-6,1e-4,1e-3],'kernel': [RBF(),Matern(),DotProduct()]}

class EBO:
    def __init__(
        self,
        pool,
        init_data,
        batch_size=30,
        iters=8,
    ) -> None:
        self.pool = deepcopy(pool)
        self.iters = iters
        self.batch_size = batch_size
        self.train_data = init_data
        self.gpr = None
        self.svr = None
        self.focused_lib = None
        self.init_data = init_data

    def explore_batch(self):
        focused_lib = generate_focused_lib(wt=wt,sites=sites,model=self.svr,num_iterations=30,num_trajectories=300)
        focused_lib = focused_lib.groupby('seq').agg('first').sort_values('y',ascending=False).reset_index()
        focused_lib = pd.merge(focused_lib,self.pool,on='seq',how='inner')
        #print("focused_lib",focused_lib.shape)
        ac = Acquisiton(focused_lib)
        #sampled_df = ac.random(sample_size=self.batch_size)
        X = np.vstack([get_esm(s) for s in focused_lib.seq.values])
        y_pre,y_std = self.gpr.predict(X,return_std=True)
        sampled_df = ac.ucb_sample(y_pre=y_pre,y_std=y_std, sample_size=self.batch_size, beta=2)
        #sampled_df = ac.pi_sample(y_pre=y_pre,y_std=y_std, sample_size=self.batch_size)
        #self.pool.drop(index=sampled_df.index, inplace=True)  # update the pool
        self.pool = self.pool[~self.pool.seq.isin(sampled_df.seq)] # update the pool
        self.pool.reset_index(drop=True, inplace=True)
        return sampled_df.reset_index(drop=True)

    def run(self):
        biggest_v_total = []
        self.fit_model()
        biggest_v_total.append(self.train_data.Fitness.values.max())
        for i in range(self.iters):
            chose_data = self.explore_batch()
            self.train_data = pd.concat(
                (self.train_data, chose_data), ignore_index=True
            )
            self.fit_model()
            biggest_v_total.append(self.train_data.Fitness.values.max())
        return biggest_v_total

    def fit_model(self):
        y = self.train_data.Fitness.values
        X = np.vstack([get_esm(s) for s in self.train_data.seq.values])
        self.gpr = fit_model(model=GaussianProcessRegressor(n_restarts_optimizer=10), X=X, y=y, param_grid=gpr_param_grid)
        self.svr = fit_model(model=SVR(), X=X, y=y, param_grid=svr_param_grid)
        return 0

In [ ]:
df_init = dftest
df_pool = df.drop(df_init.index)
ebo = EBO(pool=df_pool, init_data=df_init,iters=5,batch_size=30)
biggest_v_total = ebo.run()

In [ ]:
def topk(a, k):
    idx = np.argpartition(a, -k)[-k:]
    return idx

In [ ]:
#svr hyperparameter tuning
svr_param_grid = {'C': [.01,0.1, 1, 10, 100,500, 1000, 5000,10000],
                'gamma': [10,100,1000,500,1, 0.1, 0.01, 0.001, 0.0001]}
#gpr hyperparameter tuning
gpr_param_grid = {'alpha': [1e-10,.01,0.1, 1, 10, 1e-8,1e-6,1e-4,1e-3],'kernel': [RBF(),Matern(),DotProduct()]}

class EBO:
    def __init__(
        self,
        pool,
        init_data,
        batch_size=30,
        iters=8,
    ) -> None:
        self.pool = deepcopy(pool)
        self.iters = iters
        self.batch_size = batch_size
        self.train_data = init_data
        self.gpr = None
        self.svr = None
        self.focused_lib = None
        self.init_data = init_data

    def explore_batch(self):
        #focused_lib = generate_focused_lib(wt=wt,sites=sites,model=self.svr,num_iterations=30,num_trajectories=300)
        #focused_lib = focused_lib.groupby('seq').agg('first').sort_values('y',ascending=False).reset_index()
        #focused_lib = pd.merge(focused_lib,self.pool,on='seq',how='inner')
        #print("focused_lib",focused_lib.shape)
        X1280 = np.vstack(self.pool.esm1280.values)
        y_pre = self.svr.predict(X1280)
        #focused_lib = self.pool[y_pre < y_pre.mean()]
        focused_lib = self.pool.iloc[topk(y_pre, 80000)]
        ac = Acquisiton(focused_lib)
        sampled_df = ac.random(sample_size=self.batch_size)
        #X1280 = np.vstack(focused_lib.esm1280.values)
        #y_pre,y_std = self.gpr.predict(X1280,return_std=True)
        #sampled_df = ac.ucb_sample(y_pre=y_pre,y_std=y_std, sample_size=self.batch_size, beta=0)
        #sampled_df = ac.pi_sample(y_pre=y_pre,y_std=y_std, sample_size=self.batch_size)
        #self.pool.drop(index=sampled_df.index, inplace=True)  # update the pool
        self.pool = self.pool[~self.pool.seq.isin(sampled_df.seq)] # update the pool
        self.pool.reset_index(drop=True, inplace=True)
        return sampled_df.reset_index(drop=True)

    def run(self):
        biggest_v_total = []
        self.fit_model()
        biggest_v_total.append(self.train_data.Fitness.values.max())
        for i in range(self.iters):
            chose_data = self.explore_batch()
            self.train_data = pd.concat(
                (self.train_data, chose_data), ignore_index=True
            )
            self.fit_model()
            biggest_v_total.append(self.train_data.Fitness.values.max())
        return biggest_v_total

    def fit_model(self):
        y = self.train_data.Fitness.values
        X = np.vstack(self.train_data.esm1280.values)
        #self.gpr = fit_model(model=GaussianProcessRegressor(n_restarts_optimizer=10), X=X, y=y, param_grid=gpr_param_grid)
        self.svr = fit_model(model=SVR(), X=X, y=y, param_grid=svr_param_grid)
        return 0

In [ ]:
res = pd.DataFrame()
df_init = dftest.drop_duplicates(subset=['seq'])
df_pool = df[~df.seq.isin(df_init.seq)]
for i in range(4):
    ebo = EBO(pool=df_pool, init_data=df_init,iters=5,batch_size=50)
    biggest_v_total = ebo.run()
    res['v'+str(i)] = biggest_v_total
res.to_csv('ebo-random-5round-batchsite-50-4diff-exp.csv')